In [1]:
from lexsub.results_parser import parse_results, ttest_df, ttest_df
from lexsub.results_parser import compute_statistical_measures,  compute_statistical_measures_lr
from lexsub.results_parser import get_results_opensesame, prettify, prettify2

import os
import sys
from glob import glob
import pandas as pd
pd.set_option('max_colwidth', None)


### 1SentencePerAnnotation experiments

In [2]:
seed_exps_dir = "nExPerSent_nouns_randAllExps"
seed_exp = [(f"{seed_exps_dir}/01ExPerSent_nouns_rand01")]


expanded_exps_dir='expanded_nExPerSent_nouns_randAllExps'

expanded_exps1 = [
'01ExPerSent_nouns_rand01_expanded_lu',
'01ExPerSent_nouns_rand01_expanded_nouns-10pc',
'01ExPerSent_nouns_rand01_expanded_nouns-30pc',
'01ExPerSent_nouns_rand01_expanded_nouns-50pc',
]

expanded_exps2 = [
'01ExPerSent_nouns_rand01_expanded_roles',
'01ExPerSent_nouns_rand01_expanded_lu_roles_nouns-10pc',
'01ExPerSent_nouns_rand01_expanded_lu_roles_nouns-30pc',
'01ExPerSent_nouns_rand01_expanded_lu_roles_nouns-50pc',
]

pipeline1 = 'lugold_rolegold_nltk_nolemma_N2'
pipeline2 = 'lugold_rolegold_nltk_nolemma_role_stopwords_N2'


preds_model = 'xlnet_embs_hypers'
exps11 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps12 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps11.extend(exps12)

preds_model = 'bert'
exps21 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps22 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps21.extend(exps22)


exps = seed_exp
exps.extend(exps11)
exps.extend(exps21)


all_exps = exps
print(len(all_exps))


17


In [3]:
output_model_dir = '../parser_workdir/step_logs'

# df = get_results_opensesame(all_exps, 
#                  output_model_dir,
#                  task_name="targetid")

# df = get_results_opensesame(all_exps, 
#                  output_model_dir,
#                  task_name="frameid")

df = get_results_opensesame(all_exps, 
                 output_model_dir,
                 task_name="argid")

df = prettify(df)



columns = ['preds_model', 'pipeline', 'dataset', 'task', 'f1', 'pre', 'rec']
columns = ['preds_model', 'pipeline', 'dataset', 'task', 'f1']
# df = df[columns]
df

# check number of runs per dataset/preds_model

group_columns = ['preds_model', 'pipeline', 'dataset', 'task']
dfg=df.groupby(group_columns, as_index=False).agg(['count']).reset_index()

dfg


,preds_model,pipeline,dataset,task,exp_path,run#,f1,pre,rec
,,,,,count,count,count,count,count
0,base,,01ExPerSent-nouns-rand01,argid,10,10,10,10,10
1,bert,lugold_rolegold_nltk_nolemma_N2,augmented-lexical unit,argid,10,10,10,10,10
2,bert,lugold_rolegold_nltk_nolemma_N2,augmented-nouns-10pc,argid,10,10,10,10,10
3,bert,lugold_rolegold_nltk_nolemma_N2,augmented-nouns-30pc,argid,10,10,10,10,10
4,bert,lugold_rolegold_nltk_nolemma_N2,augmented-nouns-50pc,argid,10,10,10,10,10
5,bert,lugold_rolegold_nltk_nolemma_role_stopwords_N2,augmented-lexical unit-roles-nouns-10pc,argid,10,10,10,10,10
6,bert,lugold_rolegold_nltk_nolemma_role_stopwords_N2,augmented-lexical unit-roles-nouns-30pc,argid,10,10,10,10,10
7,bert,lugold_rolegold_nltk_nolemma_role_stopwords_N2,augmented-lexical unit-roles-nouns-50pc,argid,10,10,10,10,10
8,bert,lugold_rolegold_nltk_nolemma_role_stopwords_N2,augmented-roles,argid,10,10,10,10,10


In [ ]:
results_df = compute_statistical_measures(df)
results_df


## Visualize - Learning Curve over  F1-score

### List of paths to experiments

In [7]:
seed_exps_dir = "nPc_nouns_randAllExps"

seed_exps = """010pc_nouns
020pc_nouns
030pc_nouns
040pc_nouns
050pc_nouns
100pc_nouns""".split('\n')

seed_exps = [f"{seed_exps_dir}/{exp}" for exp in seed_exps]


expanded_exps_dir = 'expanded_nPc_nouns_randAllExps'

expanded_exps1 = """010pc_nouns_expanded_nouns-50pc
020pc_nouns_expanded_nouns-50pc
030pc_nouns_expanded_nouns-50pc
040pc_nouns_expanded_nouns-50pc
050pc_nouns_expanded_nouns-50pc
100pc_nouns_expanded_nouns-50pc""".split('\n')

expanded_exps2 = """010pc_nouns_expanded_lu_roles_nouns-50pc
020pc_nouns_expanded_lu_roles_nouns-50pc
030pc_nouns_expanded_lu_roles_nouns-50pc
040pc_nouns_expanded_lu_roles_nouns-50pc
050pc_nouns_expanded_lu_roles_nouns-50pc
100pc_nouns_expanded_lu_roles_nouns-50pc""".split('\n')

pipeline1 = 'lugold_rolegold_nltk_nolemma_N2'
pipeline2 = 'lugold_rolegold_nltk_nolemma_role_stopwords_N2'

preds_model = 'xlnet_embs_hypers'
exps11 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps12 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps11.extend(exps12)

preds_model = 'bert'
exps21 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps22 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps21.extend(exps22)

exps = seed_exps
exps.extend(exps11)
exps.extend(exps21)

# ------------------------------------------------------------------------------------------
all_exps = exps

print(len(all_exps))


30


In [9]:
output_model_dir = '../parser_workdir/step_logs'
df = get_results_opensesame(all_exps, output_model_dir)



df = prettify2(df)
# df
# columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
# group_columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size']

columns = ['preds_model', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
group_columns = ['preds_model', 'dataset', 'task', 'sample_size']

# df = df[columns]
df
# #check number of runs per dataset/preds_model
dfg=df.groupby(group_columns, as_index=False).agg(['count']).reset_index()

dfg

,preds_model,dataset,task,sample_size,exp_path,run#,f1,pre,rec,pipeline
,,,,,count,count,count,count,count,count
0,base,nPercentData,argid,10,10,10,10,10,10,10
1,base,nPercentData,argid,20,10,10,10,10,10,10
2,base,nPercentData,argid,30,10,10,10,10,10,10
3,base,nPercentData,argid,40,10,10,10,10,10,10
4,base,nPercentData,argid,50,10,10,10,10,10,10
5,base,nPercentData,argid,100,10,10,10,10,10,10
6,bert,augmented-lexical unit-roles-nouns-50pc,argid,10,10,10,10,10,10,10
7,bert,augmented-lexical unit-roles-nouns-50pc,argid,20,10,10,10,10,10,10
8,bert,augmented-lexical unit-roles-nouns-50pc,argid,30,10,10,10,10,10,10


In [6]:
results_df = compute_statistical_measures_lr(df)
results_df


preds_model                                  dataset   task  \
                                                                        
0                base                             nPercentData  argid   
1                base                             nPercentData  argid   
2                base                             nPercentData  argid   
3                base                             nPercentData  argid   
4                base                             nPercentData  argid   
5                base                             nPercentData  argid   
6                bert  augmented-lexical unit-roles-nouns-50pc  argid   
7                bert  augmented-lexical unit-roles-nouns-50pc  argid   
8                bert  augmented-lexical unit-roles-nouns-50pc  argid   
9                bert  augmented-lexical unit-roles-nouns-50pc  argid   
10               bert  augmented-lexical unit-roles-nouns-50pc  argid   
11               bert  augmented-lexical unit-roles-nouns-50pc  argid   
12  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   
13  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   
14  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   
15  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   
16  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   
17  xlnet_embs_hypers  augmented-lexical unit-roles-nouns-50pc  argid   

   sample_size     f1          pre          rec         p_value  
                 mean   std   mean   std   mean   std            
0           10  54.40  1.10  58.54  1.16  50.81  1.28  p > 0.01  
1           20  59.70  1.18  61.73  2.15  57.85  1.47  p > 0.01  
2           30  62.33  0.62  63.44  1.65  61.29  0.87  p > 0.01  
3           40  63.70  0.98  64.92  2.18  62.58  1.37  p > 0.01  
4           50  64.84  0.45  67.15  1.39  62.71  0.84  p > 0.01  
5          100  67.89  0.50  68.65  1.23  67.16  0.75  p > 0.01  
6           10  57.72  1.25  62.08  2.84  54.02  1.53  p < 0.01  
7           20  61.46  0.91  63.44  1.95  59.64  0.95  p < 0.01  
8           30  63.90  0.92  65.60  2.20  62.34  1.03  p < 0.01  
9           40  65.33  0.71  68.00  1.63  62.89  1.08  p < 0.01  
10          50  66.36  0.88  67.64  1.79  65.15  0.48  p < 0.01  
11         100  69.07  0.48  70.04  1.06  68.15  0.89  p < 0.01  
12          10  58.17  1.13  61.42  1.85  55.28  1.19  p < 0.01  
13          20  61.84  0.97  64.95  2.85  59.12  1.76  p < 0.01  
14          30  64.43  0.76  66.18  2.84  62.89  1.53  p < 0.01  
15          40  65.76  0.20  67.42  0.50  64.18  0.54  p < 0.01  
16          50  66.57  0.73  68.26  1.57  65.01  1.09  p < 0.01  
17         100  69.18  0.54  69.86  0.86  68.53  0.97  p < 0.01

In [ ]:
# ----------------------------
def results_maps(dfg, 
                 base_dataset = 'nPercentData',
                 base_model="base",
                 augmented_datasets = ['augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc'],
                 augmented_models = ['bert', 'xlnet_embs_hypers'],
                 eval_measure="f1"):

    mean_scores = {}
    std_scores = {}
    p_values = {}
#     base model scores
    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['mean'].tolist()
    mean_scores[base_model] = {base_dataset:scores}

    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['std'].tolist()
    std_scores[base_model] = {base_dataset:scores}

# augmented model scores
    for m in augmented_models:

        all_models_means = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['mean'].tolist()
            all_models_means[ds] = scores

        all_models_std = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['std'].tolist()
            all_models_std[ds] = scores

        all_models_ps = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)]['p_value'].tolist()
            all_models_ps[ds] = scores
            
        mean_scores[m] = all_models_means
        std_scores[m] = all_models_std
        p_values[m] = all_models_ps
        
    return mean_scores, std_scores, p_values



In [ ]:
mean_scores, std_scores, p_values = results_maps(results_df)
mean_scores, p_values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
%matplotlib inline

# ----------------------------------------------------------------------
N = [10,20,30,40,50, 100]

base_model = 'base'
augmented_models = ['bert', 'xlnet_embs_hypers']

datasets = ['nPercentData', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']
labels = ['seed', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']

title = "Learning Curves"
subtitles = {'bert': 'BERT',
            'xlnet_embs_hypers': 'XLNet+embs'}

colors = ['k','c', 'y', 'r', 'brown']
# ----------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, 
                         dpi=480,
                         figsize=(14, 4))

for j, m in enumerate(augmented_models):
    print(j, m)

    axes[j].set_title(f'Substitution model: {subtitles[m]}')
    axes[j].set_xlabel('Percentage of training data sampled randomly (dataset: Nouns)')
    axes[0].set_ylabel('F1-score (with gold frames)')
    axes[j].set_xticks(N)

    axes[j].set_yticks(range(50, 80, 1))
    axes[j].grid()
         
    i = -1
    for ds, label in zip(datasets, labels):
        i += 1
        print(ds, label)
        if label == 'seed': model = base_model
        else:
            model = m   
            ps = p_values[model][ds]
            # ------------------------------- to place P_values on optimally
            exp_names = list(mean_scores[model].keys())
            max_mean = [max(a,b) for a,b in zip(np.array(mean_scores[model][exp_names[0]]), np.array(mean_scores[model][exp_names[-1]]))]
            max_std = [max(a,b) for a,b in zip(np.array(std_scores[model][exp_names[0]]), np.array(std_scores[model][exp_names[-1]]))]
#             pY = [x+y for x,y in zip(max_mean, max_std)]
            pY = max_mean
        #     -------------------------------
            

        mean = np.array(mean_scores[model][ds])
        std = np.array(std_scores[model][ds])
        
        axes[j].fill_between(N, mean - std,
                             mean + std, alpha=0.1,
                             color=colors[i])

        axes[j].semilogx(N, mean, 'o-', color=colors[i],
                     label=labels[i], base=2, nonpositive='clip')
        pX = N
        p = -1
        if i==1:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 7
                    }
            for xx,yy in zip(pX,pY):
                p += 1
                #   place it below the lines
                if xx in [40, 100]:
                    axes[j].text(xx,yy-3, ps[p], fontdict=font)        
                #  place it above the lines
                else:
                    axes[j].text(xx,yy+2, ps[p], fontdict=font) 
        if i==2:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 7
                    }
            for xx,yy in zip(pX,pY):
                p += 1
                #   place it below the lines
                if xx in [40, 100]:
                    axes[j].text(xx,yy-3.75, ps[p], fontdict=font)        
                #  place it above the lines
                else:
                    axes[j].text(xx,yy+2.75, ps[p], fontdict=font) 
   

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes[:1]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
plt.figlegend( lines, labels, loc = 'lower center', ncol=3, labelspacing=0. , bbox_to_anchor=(0.5, -0.1))
# axes.legend()
plt.show()